In [60]:
import sys
sys.path.append('../FaaSterConfig')
import subprocess
import time
import os
import select
import datetime
import re

import FaaSterConfig
sys.argv = ['./FaaSterConfig.py', '../openFaas/stack.yml']

In [5]:
args, doe, stack = FaaSterConfig.main()

/Users/cameron.keenan/.pyenv/versions/FaaSterConfig/lib/python3.11/site-packages/doepy/doe_functions.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'GPU1' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.iloc[i][j]=r[j][int(df.iloc[i][j])]
/Users/cameron.keenan/.pyenv/versions/FaaSterConfig/lib/python3.11/site-packages/doepy/doe_functions.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'GPU2' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.iloc[i][j]=r[j][int(df.iloc[i][j])]
/Users/cameron.keenan/.pyenv/versions/FaaSterConfig/lib/python3.11/site-packages/doepy/doe_functions.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'GPU3' has dtype incompatible with float32, please 

In [6]:
doe

,CPU,Mem,NodeType,i,NodeTypeStr,funcName
0,0.5,248.0,0.0,1,GPU1,greeter_CPU0.5_Mem_248.0_GPU1
1,4.0,248.0,0.0,2,GPU1,greeter_CPU4.0_Mem_248.0_GPU1
2,16.0,248.0,0.0,3,GPU1,greeter_CPU16.0_Mem_248.0_GPU1
3,0.5,1024.0,0.0,4,GPU1,greeter_CPU0.5_Mem_1024.0_GPU1
4,4.0,1024.0,0.0,5,GPU1,greeter_CPU4.0_Mem_1024.0_GPU1
5,16.0,1024.0,0.0,6,GPU1,greeter_CPU16.0_Mem_1024.0_GPU1
6,0.5,16384.0,0.0,7,GPU1,greeter_CPU0.5_Mem_16384.0_GPU1
7,4.0,16384.0,0.0,8,GPU1,greeter_CPU4.0_Mem_16384.0_GPU1
8,16.0,16384.0,0.0,9,GPU1,greeter_CPU16.0_Mem_16384.0_GPU1
9,0.5,248.0,1.0,10,GPU2,greeter_CPU0.5_Mem_248.0_GPU2


In [9]:
doe.iloc[0].funcName

'greeter_CPU0.5_Mem_248.0_GPU1'

In [132]:

def read_line_with_timeout(pipe, timeout):
    rlist, _, _ = select.select([pipe], [], [], timeout)
    if rlist:
        return pipe.readline()
    else:
        return None

def run_function(funcName, stackYml=None, port=4500, MAXBUF=120):
    if not stackYml: 
        # Assume stack.yml file is same name as function
        stackYml = f"../openFaas/{funcName}.yml"

    logFile = f"/tmp/openFaas/{funcName}.log"
    os.makedirs(os.path.dirname(logFile), exist_ok=True)

    cmd = f"cd {os.path.dirname(stackYml)}; faas-cli local-run {funcName} -f {stackYml} -p {port} --quiet 2>&1 | tee {logFile}"
    print("running command: ")
    print(cmd)
    process = subprocess.Popen(cmd, bufsize=MAXBUF, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    start = datetime.datetime.now()
    duration = None
    while (datetime.datetime.now() - start).total_seconds() < 15:
    for line in process.stdout:
#     while True:
        line = read_line_with_timeout(process.stdout, .1)
        if line:
            print(line)
            if re.search("Listening on port: 8080", line):
                # TODO use requests and send payload from input 
                os.system(f"curl http://0.0.0.0:{port} -X POST  --data '5000'")
            elif (match:= re.search("Duration: ((?:\d|.)+)s", line)):
                duration = float(match.groups()[0])
                break
                
    if not process.poll():
        process.kill()
    # Ensure all subprocesses are killed for this function
    os.system("""kill -9 $(ps -ef | grep -e "faas-cli local-run %s" -e "docker run --name %s" -e "run.*%s" | awk '{print $2}')""" % (funcName,funcName,funcName))

    return duration

IndentationError: expected an indented block after 'while' statement on line 23 (665274702.py, line 24)

In [133]:
duration = run_function('matmul')
print(f"result: {duration}")

running command: 
cd ../openFaas; faas-cli local-run matmul -f ../openFaas/matmul.yml -p 4500 --quiet 2>&1 | tee /tmp/openFaas/matmul.log
Building: matmul

[0] > Building matmul.

Building: matmul:latest with python3 template. Please wait..

Image: matmul:latest built.

[0] < Building matmul done in 1.28s.

[0] Worker done.



Total build time: 1.28s

Image: matmul:latest

Starting local-run for: matmul on: http://0.0.0.0:4500



2023/10/24 12:54:14 Version: 0.2.2	SHA: f997bb97fea3a3196eb054994f2d44b038b297c2

2023/10/24 12:54:14 Timeouts: read: 30s write: 30s hard: 0s health: 30s.

2023/10/24 12:54:14 Listening on port: 8080



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     4    0     0  100     4      0      1  0:00:04  0:00:02  0:00:02     1

1.6444599628448486
1.6444599628448486
2023/10/24 12:54:14 Writing lock-file to: /tmp/.lock

2023/10/24 12:54:14 Metrics listening on port: 8081

2023/10/24 12:54:14 Forking fprocess.

2023/10/24 12:54:17 Wrote 38 Bytes - Duration: 2.480701s

result: 2.480701


100    42  100    38  100     4     15      1  0:00:04  0:00:02  0:00:02    16


In [101]:
duration

2.859549